# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 3 2022 2:13PM,238794,19,DEX0006941,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
1,Jun 3 2022 2:13PM,238794,19,DEX0006942,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jun 3 2022 2:13PM,238794,19,DEX0006943,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jun 3 2022 2:13PM,238794,19,DEX0006944,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jun 3 2022 2:13PM,238794,19,DEX0006945,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jun 3 2022 2:13PM,238794,19,DEX0006946,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,Jun 3 2022 2:13PM,238794,19,DEX0006947,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
7,Jun 3 2022 2:13PM,238794,19,DEX0006948,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
8,Jun 3 2022 2:13PM,238794,19,DEX0006949,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
9,Jun 3 2022 2:13PM,238794,19,DEX0006950,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,238790,Released,1
31,238791,Released,1
32,238792,Released,44
33,238793,Released,1
34,238794,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238790,NaN,NaN,1.0
238791,NaN,NaN,1.0
238792,NaN,NaN,44.0
238793,NaN,NaN,1.0
238794,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238711,0.0,0.0,1.0
238713,2.0,14.0,2.0
238720,0.0,0.0,1.0
238721,0.0,0.0,1.0
238748,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238711,0,0,1
238713,2,14,2
238720,0,0,1
238721,0,0,1
238748,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238711,0,0,1
1,238713,2,14,2
2,238720,0,0,1
3,238721,0,0,1
4,238748,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238711,,,1
1,238713,2,14,2
2,238720,,,1
3,238721,,,1
4,238748,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 3 2022 2:13PM,238794,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
10,Jun 3 2022 2:09PM,238792,10,ISDIN Corporation
54,Jun 3 2022 2:06PM,238783,10,ISDIN Corporation
99,Jun 3 2022 2:05PM,238793,10,ISDIN Corporation
100,Jun 3 2022 2:01PM,238791,18,"ClearSpec, LLC"
101,Jun 3 2022 1:59PM,238790,18,"ClearSpec, LLC"
102,Jun 3 2022 1:51PM,238789,18,"ClearSpec, LLC"
103,Jun 3 2022 1:45PM,238788,19,ACG North America LLC
104,Jun 3 2022 1:40PM,238787,16,"SHL Pharma, LLC"
105,Jun 3 2022 1:33PM,238786,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 3 2022 2:13PM,238794,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,10
1,Jun 3 2022 2:09PM,238792,10,ISDIN Corporation,,,44
2,Jun 3 2022 2:06PM,238783,10,ISDIN Corporation,,,45
3,Jun 3 2022 2:05PM,238793,10,ISDIN Corporation,,,1
4,Jun 3 2022 2:01PM,238791,18,"ClearSpec, LLC",,,1
5,Jun 3 2022 1:59PM,238790,18,"ClearSpec, LLC",,,1
6,Jun 3 2022 1:51PM,238789,18,"ClearSpec, LLC",,,1
7,Jun 3 2022 1:45PM,238788,19,ACG North America LLC,,,1
8,Jun 3 2022 1:40PM,238787,16,"SHL Pharma, LLC",,,1
9,Jun 3 2022 1:33PM,238786,20,Alumier Labs Inc.,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 2:13PM,238794,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10,,
1,Jun 3 2022 2:09PM,238792,10,ISDIN Corporation,44,,
2,Jun 3 2022 2:06PM,238783,10,ISDIN Corporation,45,,
3,Jun 3 2022 2:05PM,238793,10,ISDIN Corporation,1,,
4,Jun 3 2022 2:01PM,238791,18,"ClearSpec, LLC",1,,
5,Jun 3 2022 1:59PM,238790,18,"ClearSpec, LLC",1,,
6,Jun 3 2022 1:51PM,238789,18,"ClearSpec, LLC",1,,
7,Jun 3 2022 1:45PM,238788,19,ACG North America LLC,1,,
8,Jun 3 2022 1:40PM,238787,16,"SHL Pharma, LLC",1,,
9,Jun 3 2022 1:33PM,238786,20,Alumier Labs Inc.,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 2:13PM,238794,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10,,
1,Jun 3 2022 2:09PM,238792,10,ISDIN Corporation,44,,
2,Jun 3 2022 2:06PM,238783,10,ISDIN Corporation,45,,
3,Jun 3 2022 2:05PM,238793,10,ISDIN Corporation,1,,
4,Jun 3 2022 2:01PM,238791,18,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 2:13PM,238794,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10.0,NaN,NaN
1,Jun 3 2022 2:09PM,238792,10,ISDIN Corporation,44.0,NaN,NaN
2,Jun 3 2022 2:06PM,238783,10,ISDIN Corporation,45.0,NaN,NaN
3,Jun 3 2022 2:05PM,238793,10,ISDIN Corporation,1.0,NaN,NaN
4,Jun 3 2022 2:01PM,238791,18,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 2:13PM,238794,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10.0,0.0,0.0
1,Jun 3 2022 2:09PM,238792,10,ISDIN Corporation,44.0,0.0,0.0
2,Jun 3 2022 2:06PM,238783,10,ISDIN Corporation,45.0,0.0,0.0
3,Jun 3 2022 2:05PM,238793,10,ISDIN Corporation,1.0,0.0,0.0
4,Jun 3 2022 2:01PM,238791,18,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2387618,148.0,14.0,2.0
102,477554,2.0,0.0,0.0
12,716308,3.0,0.0,0.0
15,238755,1.0,0.0,0.0
16,477535,2.0,0.0,0.0
18,1193811,5.0,0.0,0.0
19,716356,13.0,0.0,0.0
20,716335,33.0,19.0,0.0
22,477559,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2387618,148.0,14.0,2.0
1,102,477554,2.0,0.0,0.0
2,12,716308,3.0,0.0,0.0
3,15,238755,1.0,0.0,0.0
4,16,477535,2.0,0.0,0.0
5,18,1193811,5.0,0.0,0.0
6,19,716356,13.0,0.0,0.0
7,20,716335,33.0,19.0,0.0
8,22,477559,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,148.0,14.0,2.0
1,102,2.0,0.0,0.0
2,12,3.0,0.0,0.0
3,15,1.0,0.0,0.0
4,16,2.0,0.0,0.0
5,18,5.0,0.0,0.0
6,19,13.0,0.0,0.0
7,20,33.0,19.0,0.0
8,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,148.0
1,102,Released,2.0
2,12,Released,3.0
3,15,Released,1.0
4,16,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,22
Status,,,,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,14.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0
Released,148.0,2.0,3.0,1.0,2.0,5.0,13.0,33.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0
2,Released,148.0,2.0,3.0,1.0,2.0,5.0,13.0,33.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0
2,Released,148.0,2.0,3.0,1.0,2.0,5.0,13.0,33.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()